In [ ]:
import webhoseio, json
import urllib, sys, os, requests
from urllib.request import urlopen
from urllib.parse import urlencode


ArticleListFile = 'f:\\anly540.json'
url= "f:\\TextToProcess.txt"

print('Starting Program')

In [ ]:

webhoseio.config(token="a7e5472d-d08b-478e-915a-d036a99f6712")

query_params = { "q": "organization:Tesla", "ts": "1509161113588", "sort": "crawled" }

print('Querying data')
output = webhoseio.query("filterWebContent", query_params)

print('Data quieries: ')

print(len(output['posts']))

    
#Get the next batch of posts
x=0
#Each query is giving 100 feeds. for 900, the loop runs 9 times.
while len(output['posts'])<1000:
    output1 = webhoseio.get_next()
    x=x+1
    print(str(x) + ': '+ str(len(output1['posts'])))
    for itm in output1['posts']:
        output['posts'].append(itm)

print('loop complete')

outfile = open(ArticleListFile, 'w')

print('Saving file')

json.dump(output, outfile)

print('file saved')

outfile.close()

print('End program')

In [ ]:
print('Reading file')
outfileRead=open(ArticleListFile, 'r')

print('Getting data in dictionary')
data=json.load(outfileRead)

TextToProcess=''

for y in range(0, 999):
    TextToProcess = TextToProcess + data['posts'][y]['title'] + '\n'


f = open(url,'wb')
f.write(str.encode(TextToProcess))
f.close()

In [ ]:
file= open(url,'rb')

    
#Calais API key.
x_ag_access_token1 = 'bq9XnoneGq87PB5yxjjDZdG3GpmuxfUC' 

#REST URL of CAlais.
calaisREST_URL = 'https://api.thomsonreuters.com/permid/calais' 

##### send data to Calais API.
header = {'X-AG-Access-Token' : x_ag_access_token1, 'Content-Type' : 'text/plain', 'outputformat' : 'application/json'}

print("Sending")
##### get API results and print them.
results =  requests.post(calaisREST_URL, data=file, headers=header)
print (results)
print (results.text)

output_file_name = os.path.basename('Entities') + '.xml'
output_file = open(os.path.join('f:\\', output_file_name), 'wb')
output_file.write(results.text.encode('utf-8'))
output_file.close()

file.close()
    
print("done")

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import nltk, re
from nltk.stem.wordnet import WordNetLemmatizer

stopwords = set(nltk.corpus.stopwords.words('english'))

def tokenize(text):
    # first tokenize by sentence, then by word to ensure that
    # punctuation is caught as it's own token

    tokens  = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent) if word not in stopwords]
    lmtzr   = WordNetLemmatizer()
    
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token.lower())
            
    bigrams  = [' '.join(bigram) for bigram in nltk.bigrams(filtered_tokens)]
    trigrams = [' '.join(trigram) for trigram in nltk.trigrams(filtered_tokens)]
    stems = [lmtzr.lemmatize(t,'v') for t in filtered_tokens]
    
    for bigram in bigrams:
        if bigram not in stopwords:
            stems.append(bigram)
            
    for trigram in trigrams:
        if trigram not in stopwords:
            stems.append(trigram)
            
    return stems



def clstr_lda(num_topics, stories):
    """
    
    :rtype: object
    """
    
    n_top_words = 10
    
    tf_vectorizer = CountVectorizer(max_df=0.8, min_df=0.2, max_features=1000,tokenizer=tokenize, ngram_range=(1,3))
    tf = tf_vectorizer.fit_transform(stories)
    lda = LatentDirichletAllocation(n_components=num_topics, max_iter=200, learning_method='online', learning_offset=10.,random_state=1)
    lda.fit(tf)
    tf_feature_names = tf_vectorizer.get_feature_names()
    
    #print("\nLDA Topics:")
    # print top topic words
    
    topics = dict()
    for topic_idx, topic in enumerate(lda.components_):
        topics[topic_idx] = [tf_feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]
        print("Topic #%d:" % topic_idx)
        print(" | ".join([tf_feature_names[i]
                          for i in topic.argsort()[:-n_top_words - 1:-1]]))
        #print()
    return topics



if __name__ == '__main__':
    myinput = []
    
    print('Reading file')
    outfileRead=open('F:\\anly540.json', 'r')
    
    print('Getting data in dictionary')
    
    data=json.load(outfileRead)
    
    for y in range(0, 999):
        myinput.append(data['posts'][y]['text'])


topics = clstr_lda(100, myinput)

import csv

with open('f:\\topics.csv', 'w') as csv_file:
    writer = csv.writer(csv_file)
    for key, value in topics.items():
       writer.writerow([key, value])



In [6]:
import gensim, operator
from scipy import spatial
import numpy as np
from gensim.models import KeyedVectors

model_path = 'C:/Users/Rahul/OneDrive/Harrisburg/ANLY 540/Assignment 6/'

In [7]:
def load_wordvec_model(modelName, modelFile, flagBin, limit):
    print('Loading ' + modelName + ' model...')
    
    if limit > 0:
        model = KeyedVectors.load_word2vec_format(model_path + modelFile, binary=flagBin, limit=limit)
    else:
        model = KeyedVectors.load_word2vec_format(model_path + modelFile, binary=flagBin)
    
    print('Finished loading ' + modelName + ' model...')
    return model

model_word2vec = load_wordvec_model('Word2Vec', 'GoogleNews-vectors-negative300.bin.gz', True, 500000)

Loading Word2Vec model...
Finished loading Word2Vec model...


In [8]:
# develop own topic taxonomy
topic_taxonomy = {
        "time chronology measurement":
        {
            "chonology": "today back future continue ago",
            "day component": "october wednesday year september november", 
            "time scale": "delay period quarter months week"
        },
        "product making":
        {
            "product manufacturing": "production produce build manufacture", 
            "product categorization": "make model", 
            "creation": "new become increase",
            "introduction": "release launch"
        },
        "numeric sequence":
        {
            "number": "one two three",
            "scales": "million billion", 
            "sequential": "second first last next recent current"
        },
        "object transfer":
        {
            "object handover": "give provide sell push", 
            "object arrangement": "set trade sales offer", 
            "Object takeover": "get take use buy earn"
        },
        "Business company":
        {
            "Business name": "tsla tesla", 
            "Business type": "company firm industry business",
            "Business stakeholder": "CEO lead invester people world analysts", 
            "Business stakes": "market share investment stock loss hold"
        },
        "Technology object manipulation":
        {
            "Device feature": "technology system", 
            "components and energy": "electric battery power charge energy", 
            "manipulation": "go start operate work move"
        }
} 



In [9]:
def vec_similarity(input1, input2, vectors):
    term_vectors = [np.zeros(300), np.zeros(300)]
    terms = [input1, input2]
        
    for index, term in enumerate(terms):
        for i, t in enumerate(term.split(' ')):
            try:
                term_vectors[index] += vectors[t]
            except:
                term_vectors[index] += 0
        
    result = (1 - spatial.distance.cosine(term_vectors[0], term_vectors[1]))
    if result is 'nan':
        result = 0
        
    return result

In [10]:
# function checks whether the input words are present in the vocabulary for the model
def vocab_check(vectors, words):
    
    output = list()
    for word in words:
        if word in vectors.vocab:
            output.append(word.strip())
            
    return output

In [11]:
# function calculates similarity between two strings using a particular word vector model
def calc_similarity(input1, input2, vectors):
    s1words = set(vocab_check(vectors, input1.split()))
    s2words = set(vocab_check(vectors, input2.split()))
    
    output = vectors.n_similarity(s1words, s2words)
    return output

In [12]:
# function takes an input string, runs similarity for each item in topic_taxonomy, sorts and returns top 3 results
def classify_topics(input, vectors):
    feed_score = dict()
    for key, value in topic_taxonomy.items():
        max_value_score = dict()
        for label, keywords in value.items():
            max_value_score[label] = 0
            topic = (key + ' ' + keywords).strip()
            max_value_score[label] += float(calc_similarity(input, topic, vectors))
            
        sorted_max_score = sorted(max_value_score.items(), key=operator.itemgetter(1), reverse=True)[0]
        feed_score[sorted_max_score[0]] = sorted_max_score[1]
    return sorted(feed_score.items(), key=operator.itemgetter(1), reverse=True)[:3]

In [14]:
if __name__ == '__main__':
    # example using Word2Vec
    output1 = classify_topics('Tesla Company CEO Elon Musk is considered visionary and young entrepreneur idol', model_word2vec)
    print(output1)
    

[('Business stakeholder', 0.5232430912801942), ('product categorization', 0.3393615919584506), ('components and energy', 0.30000098325734914)]
